In [37]:
!pip install PrettyTable --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [438]:
import io, os
import re as re
import zipfile as zipfile
import time
import math
import numpy as np
from operator import itemgetter
from prettytable import PrettyTable

In [439]:
textsWithTextNumbers = {}       # { doc name(number) : text }

mytextzip = ''
path = './30Columnists.zip'

with zipfile.ZipFile(path) as z:
    for zipinfo in z.infolist():
        if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
            with z.open(zipinfo) as f:
                textfile = io.TextIOWrapper(f, encoding='cp1254', newline='')
                for line in textfile:
                    if len(line.strip()): mytextzip += line.strip()

                filename = str(zipinfo.filename)
                filename = filename.split("/")
                filename = filename[-1][:-4]
                
                textsWithTextNumbers[filename] = mytextzip
                mytextzip = ''

In [440]:
stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [441]:
def Tokenizer(text, stop):
    text = (str(text)).lower()          # normalize
    words = re.findall(r'\w+', text)    # find all words in it

    if stop == 1:
        #remove stopwords, start last word to avoid index problems in loop
        for x in range(len(words)-1, -1, -1):
            word = str(words[x])
            if word in stopwords:
                words.pop(x)
            elif word.lower() in stopwords:
                words.pop(x)
    
    return words

In [442]:
docsWords = {}      # { doc name(number) : words in it }

def turner(textsWithTextNumbers):
    temp = {}
    for doc, text in textsWithTextNumbers.items():
        temp[doc] = Tokenizer(text, 1)
    return temp

In [443]:
docsWords = turner(textsWithTextNumbers)

In [444]:
# database to keep words with docs where they are in
db = {}    # "42" : ['1.1', '1.32', ...]
db_ = {}     # "42" : [('1.1', 1),...}

for doc, words in docsWords.items():
    words = sorted(words)
    n = 1
    for i, word in enumerate(words):
        if (i+1) != len(words):
            nextw = words[i+1]
            if word == nextw:
                n += 1
                continue
        if word not in db:
            db[word] = []
            db_[word] = []
        db[word].append(doc)
        db_[word].append((doc, n))

        n = 1

In [445]:
term_frequencies = {}       # { doc: {word: word frequency} }
                            # e.g. '1.1': [['doubt', 0.003745318352059925],...]

for doc, words in docsWords.items():
    d = {}
    temp = []
    
    #term frequency
    for word in words:
        if word not in d:
            d[word] = 1
        else:
            d[word] += 1

    n = len(d)      # of words in doc
    for word, number in d.items():
        tf = number / n
        temp.append([word, tf])

    term_frequencies[doc] = temp

In [446]:
idfs = {}            # {word: number} ---> {word: idf}
                     # e.g. {'doubt': 3.1958930192167068}

for doc, array in term_frequencies.items():
    for word, tf in array:
        if word not in idfs:
            idfs[word] = 1
        else:
            idfs[word] += 1

doc_total = len(term_frequencies)
for word, n in idfs.items():
    idf = ((1 + doc_total) / (n+1))
    idfs[word] = math.log(idf) + 1

In [447]:
# TF-IDF = TF * IDF

tfidf = {}
#{doc: {word: tf-idf}}
# e.g. '1.11': {'time': 0.009378893551214249, 'year': 0.02088227471955536,...} 
# e.g. '1.12': {'time': 0.010133140235402807, 'year': 0.011280809245354205,...}

for doc, array in term_frequencies.items():
    tfidf[doc] = {}
    for word, tf in array:
        TfIdf = tf * idfs[word]
        tfidf[doc][word] = TfIdf

In [448]:
lens = [len(words) for words in term_frequencies.values()]
avgN = sum(lens) / len(lens)

## Okapi BM25
BM25 Algorithm is  
sum of ((IDF for word in query) times  
((frequency of the word in document) times (@paramK+1)) over  
((frequency of the word in document + @paramK) times ((1 - @paramB) + (b times (length of document over avg length of document)))

In [449]:
def calc_BM25(IDF, TF, K, B, N, avgN):
    first = TF * (K + 1)
    second = TF + (K * ((1 - B) + (B * (N / avgN))))
    return IDF * (first / second)

In [450]:

# IDF = idfs['doubt']
# TF = term_frequencies['1.1'][0][1]

# N = len(term_frequencies['1.1'])
# lens = [len(words) for words in term_frequencies.values()]
# avgN = sum(lens) / len(lens)

# calc_BM25(IDF, TF, K, B, N, avgN)

In [451]:
from collections import defaultdict
BM = defaultdict(lambda: defaultdict(int))
#{doc: {word: bm}}
lens = [len(words) for words in term_frequencies.values()]
avgN = sum(lens) / len(lens)
K = 1.6
B = 0.75
for doc in term_frequencies:
    N = len(term_frequencies[doc])
    for idx, word in enumerate(term_frequencies[doc]):
        TF = word[1]
        IDF = idfs[word[0]]
        BM[doc][word[0]] = calc_BM25(IDF, TF, K, B, N, avgN)

### DFI
e = frequency of the word in corpus times length of the document over number of documents

DFI = log((frequency of the word in the document - e) over e squared) on base 2

TF = total frequency of the word in corpus  

tf = frequency of the word in document

In [452]:
import math

def calc_DFI(TF,tf, n, N):
    e = (TF * n) / N
    return log( ( ( tf - e ) / math.sqrt(e) ) + 1 )
    

In [453]:
from collections import defaultdict
DFI = defaultdict(lambda: defaultdict(int))
TFDict = defaultdict()
N = sum([len(term_frequencies[doc]) for doc in term_frequencies])


for doc in term_frequencies:
    for termTF in term_frequencies[doc]:
        if termTF[0] in TFDict:
            TFDict[termTF[0]] += termTF[1]
        else:
            TFDict[termTF[0]] = termTF[1]

            
for doc in term_frequencies:
    n = len(term_frequencies[doc])
    for termTF in term_frequencies[doc]:
        tf = termTF[1]
        TF = TFDict[termTF[0]]
        DFI[doc][termTF[0]] = calc_DFI(TF, tf, n, N)

In [454]:
def clusterDB(entry):
    NinDB = []      # words in entry but not in DB
    inDB = []       # words in entry and DB

    # cluster words in entry
    for word in entry:
        word = word.lower()
        if word not in db and word not in NinDB:
            NinDB.append(word)
        elif word in db and word not in inDB:
            inDB.append(word)
    
    return inDB, NinDB

In [455]:
def findDualTuples(inDB):
    docs = {}       # words tuple: common documents where they are              # ('gold', 'rich'): ['26.12', '15.50']
    w = []          # words tuples                                              # [('gold', 'rich'), ..., ('gold', 'man', 'rich')]

    temp = inDB.copy()     # for inner loop -> remove words itself every main loop

    # compare words documents where they are to find common docs
    # just for find dual tuples
    for word1 in inDB:
        temp.remove(word1)
        for word2 in temp:
            join = list(set(db[word1]) & set(db[word2]))
            if len(join) > 0:
                w.append((word1, word2))
                docs[(word1, word2)] = join

    return docs, w

In [456]:
def findAllIntersections(docs, w):
    # detect common docs between every words
    # finds all sub and main intersections
    for i, tuple1 in enumerate(w):
        
        # there is no second tuple to find commons
        if (len(w)-1) == i:
            break

        # for inner loop -> don't need to look itself and before tuples
        inner = w[(i+1):]

        # tuples that have same length can be compared because of hierarchy
        l1 = len(tuple1)
        for tuple2 in inner:
            l2 = len(tuple2)
            if l1 != l2:
                continue

            # tuples(words) are combined
            n = tuple1 + tuple2
            # to avoid being listed separate classification of different rankings of the same words
            n = sorted(n)
            # to avoid using the same word more than once
            n = tuple(dict.fromkeys(n))
            # find common docs
            join = list(set(docs[tuple1]) & set(docs[tuple2]))

            # there is at least one common document and group of same words were not found before by different tuple pairs
            if len(join) > 0 and n not in w:
                w.append(n)
                docs[n] = join

    return docs

In [457]:
def fitResults(docs, mod):
    # keep related keys, docs and tf-idfs in same index
    x = []      # [doc]
    y = []      # [keys]
    z = []      # [total tfidfs]

    for tuplex, docs in docs.items():      # tuplex -> word tuples, docs -> keys common docs
        for doc in docs:
            t = 0
            for word in tuplex:
                if mod == "tfidf":
                    t += tfidf[doc][word]   # word's tf-idf value in doc
                                            # sum of values represent relation between tuple(words) and doc
                elif mod == "bm25":
                    t+= BM[doc][word]
                else:
                    t+= DFI[doc][word]
                
            
            # to check whether the value has been calculated beforehand for this doc
            if doc in x:
                i = x.index(doc)
                if z[i] > t:     # this doc was associated before with better tuple
                    continue
                else:            # this tuple better, delete old one
                    x.pop(i)
                    y.pop(i)
                    z.pop(i)
            # add same index
            x.append(doc)       # doc
            y.append(tuplex)    # tuple
            z.append(t)         # sum of tf-idf values of words for this doc

    return x, y, z

In [458]:
# number of terms important
def sortResults(x, y, z):
    r = []
    for i in range(len(x)):
        # take max relavent index
        maxIdx = np.argmax(z)
        
        # add result to results
        a = [x[maxIdx], str(y[maxIdx]), z[maxIdx], len(y[maxIdx])]
        
        # delete until there is no result
        x.pop(maxIdx)
        y.pop(maxIdx)
        z.pop(maxIdx)

        r.append(a)

    temp = {}
    for xx, yy, zz, ll in r:
        if ll not in temp.keys():
            temp[ll] = []
        temp[ll].append([xx, yy, zz])

    temp_items = temp.items()
    sorted_items = sorted(temp_items, reverse= True)

    results = []
    for key, part in sorted_items:
        for a in part:
            results.append(a)

    return results

In [459]:
# 1-1
def OneByOne(wordsInDB, mod):
    x = []  # docs
    y = []  # terms
    z = []  # tfidfs
    
    for term in wordsInDB:
        docus = db[term]
        for doc in docus:
            if mod == "tfidf":
                v = tfidf[doc][term]
            elif mod == "bm25":
                v = BM[doc][term]
            else:
                v = DFI[doc][term]
            
            x.append(doc)
            y.append(term)
            z.append(v)

    r = []
    for i in range(len(x)):
        # take max relavent index
        maxIdx = np.argmax(z)
        # add result to results
        a = [x[maxIdx], str(y[maxIdx]), z[maxIdx]]
        # delete until there is no result
        x.pop(maxIdx)
        y.pop(maxIdx)
        z.pop(maxIdx)

        r.append(a)

    return r

In [460]:
def Search(entry, mod):
    entry = Tokenizer(entry, 0)

    # cluster words in entry
    NinDB = []      # words in entry but not in DB
    inDB = []       # words in entry and DB
    inDB, NinDB = clusterDB(entry)

    # warning message for words that not in DB
    if len(inDB) == 0:
        print("\nThere is no doc that include these terms. Try again.")
        return
    if len(NinDB) != 0:
        p = " - "
        warning = p.join(NinDB)
        print("\n'" + warning + "' -> not in DB")

    # there is just one word
    if len(inDB) == 1:
        return OneByOne(inDB, mod)

    elif len(inDB) > 1:
        # find dual tuples
        docs = {}       # words tuple: common documents where they are              # ('gold', 'rich'): ['26.12', '15.50']
        w = []          # words tuples                                              # [('gold', 'rich'), ..., ('gold', 'man', 'rich')]    
        docs, w = findDualTuples(inDB)
        
        # no common docs between every dual words documents
        if len(w) == 0:
            print("\nNo common documents!")
            return OneByOne(inDB, mod)

        # detect common docs between every words
        # finds all sub and main intersections
        docs = findAllIntersections(docs, w)

        # keep related keys, docs and tf-idfs in same index
        x, y, z = fitResults(docs, mod)

        return sortResults(x, y, z)

In [461]:
def printResults(r):
    print("\n"+str(len(r)) + " search results were found." )
    t = PrettyTable(["Doc", "Terms", "Relevance"])

    for i, result in enumerate(r):
        t.add_row(result)
    print(t)

In [463]:
mod = str(input('Select your preffered model: tfidf / bm25 / dfi'))
print(mod)
entry = str(input('Enter a query: '))
results = Search(entry, mod)
if isinstance(results, list):
    printResults(results)
res = []
for i in range(10):
    res.append(results[i][0])
res

# Global warming
# tfidf -> 10.37, 30.36, 20.21 10.4 30.49 5.34 5.44 5.11 7.27 5.30 5.17 2.4
# bm25 -> 30.36 30.49 20.21 10.37 5.34 10.4 5.44 5.11 7.27  5.30  5.17 2.4 
# dfi -> '30.36','10.37','30.49','20.21','5.34','5.44','5.11','10.4','7.27','5.30'

# champions league
# tfidf -> '12.11','12.8',12.7','8.29','28.14','8.17','12.37','28.50','14.33','26.5'
# bm25 -> '12.7','12.11','28.14','14.33','8.17','12.8','28.50','28.43','28.25','28.5'
# dfi -> '12.7','12.11','28.14','12.8','14.33','8.17','28.43','28.50','28.25','12.37'

# racism
# tfidf -> '3.3','4.35','30.5','5.46','13.47','13.48','3.30','4.9','26.42','4.38'
# bm25 -> '3.3','30.5','4.35','5.46','13.47','13.48','3.30','4.9','26.42','4.38'
# dfi -> '3.3','30.5','4.35','5.46','13.47','13.48','3.30','4.9','26.42','4.38'

# alcohol and drug abuse
# tfidf -> '23.20','16.22','4.9','3.1','11.50','8.36','8.20','15.10','11.23','16.44'
# bm25 -> '23.20','16.22','4.9','3.1','11.50','8.36','8.20','11.23','16.44','15.10'
# dfi -> '23.20','16.22','4.9','11.50','3.1','8.36','8.20','11.23','3.28','15.10'

# exercising for kids
# tfidf -> '23.26','23.40','23.29','23.35','23.44','23.14','23.28','23.38','23.43','23.10'
# bm25 -> '23.26','23.40','23.29','23.35','23.44','23.28','23.14','23.38','23.43','23.10'
# dfi -> '23.40','23.26','23.29','23.35','23.28','23.44','23.14','23.38','23.10','23.43'

# obama vs clinton election
# tfidf -> '4.7','5.35','5.24','10.29','1.26','10.17','5.42','10.15','10.18','10.11'
# bm25 -> '4.7','5.35','5.24','1.26','5.42','5.48','10.17','10.29','10.15','5.27'
# dfi -> '4.7','5.24','5.35','1.26','5.42','5.48','10.17','10.43','5.27','10.18'

# health insurance
# tfidf -> '24.31','24.32','25.41','24.29','25.9','25.16','25.27','25.48','25.7','25.17'
# bm25 -> '24.32','24.31','25.41','25.9','24.29','25.48','25.8','25.39','25.16','25.27'
# dfi -> '24.32','24.31','24.29','25.41','25.9','25.8','25.48','25.27','25.7','24.26'

# breast cancer risk for women
# tfidf -> '24.48','24.49','24.50','24.47','24.23','24.6','24.7','9.37','24.2','11.35'
# bm25 -> '24.48','24.49','24.50','24.23','24.47','11.35','24.7','24.6','9.37','24.3'
# dfi -> '24.48','24.49','24.50','24.23','24.7','9.37','24.47','11.35','24.2','24.3'

# cancer risk for smokers
# tfidf -> '24.7','27.35','24.34','24.3','24.48','24.49','24.50','24.2','27.19','9.50'
# bm25 -> '24.3','24.7','27.35','24.34','29.40','24.39','11.35','24.6','24.48','24.49'
# dfi -> '24.3','24.7','27.35','29.40','24.34','24.39','11.35','24.48','24.49','24.50'

# alzheimer risk
# tfidf -> '27.28','11.47','9.30','11.29','11.50','27.47','9.27','11.17','11.25','9.33'
# bm25 -> '27.28','9.30','11.47','11.29','11.17','9.27','11.50','27.47','9.33','11.25'
# dfi -> '27.28','11.47','9.30','11.29','11.17','11.50','9.27','27.47','9.33','11.25'

Select your preffered model: tfidf / bm25 / dfidfi
dfi
Enter a query: breast cancer risk for women

'for' -> not in DB

123 search results were found.
+-------+---------------------------------------+-----------------------+
|  Doc  |                 Terms                 |       Relevance       |
+-------+---------------------------------------+-----------------------+
| 24.48 | ('breast', 'cancer', 'risk', 'women') |   5.456583863614265   |
| 24.49 | ('breast', 'cancer', 'risk', 'women') |   5.456583863614265   |
| 24.50 | ('breast', 'cancer', 'risk', 'women') |   5.456583863614265   |
| 24.23 | ('breast', 'cancer', 'risk', 'women') |    5.04164142544976   |
|  24.7 | ('breast', 'cancer', 'risk', 'women') |   4.044986118636013   |
|  9.37 | ('breast', 'cancer', 'risk', 'women') |   3.9894709400103903  |
| 24.47 | ('breast', 'cancer', 'risk', 'women') |    3.97902997972275   |
| 11.35 | ('breast', 'cancer', 'risk', 'women') |   3.8764694279172325  |
|  24.2 | ('breast', 'cancer', 'ris

['24.48',
 '24.49',
 '24.50',
 '24.23',
 '24.7',
 '9.37',
 '24.47',
 '11.35',
 '24.2',
 '24.3']

In [464]:
okapiQueries = defaultdict(list)
tfidfQueries = defaultdict(list)
dfiQueries = defaultdict(list)

tfidfQueries['global warming'] = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
okapiQueries['global warming'] = [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
dfiQueries['global warming'] =   [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

tfidfQueries['champions league'] = [1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
okapiQueries['champions league'] = [1, 1, 1, 0, 1, 1, 0, 0, 0, 0]
dfiQueries['champions league'] =   [1, 1, 1, 1, 0, 1, 0, 0, 0, 1]

tfidfQueries['racism'] = [1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
okapiQueries['racism'] = [1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
dfiQueries['racism'] =   [1, 1, 1, 0, 0, 1, 0, 0, 0, 0]

tfidfQueries['alcohol and drug abuse'] = [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
okapiQueries['alcohol and drug abuse'] = [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
dfiQueries['alcohol and drug abuse'] =   [1, 1, 0, 0, 1, 0, 0, 0, 0, 0]

tfidfQueries['exercising for kids'] = [1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
okapiQueries['exercising for kids'] = [1, 1, 1, 1, 1, 0, 1, 1, 1, 0]
dfiQueries['exercising for kids'] =   [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]

tfidfQueries['obama vs clinton election'] = [1, 1, 0, 1, 1, 0, 1, 0, 0, 0]
okapiQueries['obama vs clinton election'] = [1, 1, 1, 1, 1, 1, 0, 1, 0, 1]
dfiQueries['obama vs clinton election'] =   [1, 1, 1, 1, 1, 1, 0, 1, 1, 0]

tfidfQueries['health insurance'] = [1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
okapiQueries['health insurance'] = [1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
dfiQueries['health insurance'] =   [1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

tfidfQueries['breast cancer risk for women'] = [1, 1, 1, 1, 1, 1, 0, 1, 0, 0]
okapiQueries['breast cancer risk for women'] = [1, 1, 1, 1, 1, 0, 0, 1, 0, 0]
dfiQueries['breast cancer risk for women'] =   [1, 1, 1, 1, 0, 1, 1, 0, 0, 0]

tfidfQueries['cancer risk for smokers'] = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
okapiQueries['cancer risk for smokers'] = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
dfiQueries['cancer risk for smokers'] =   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


tfidfQueries['alzheimer risk'] = [1, 1, 1, 1, 1, 0, 1, 0, 0, 0]
okapiQueries['alzheimer risk'] = [1, 1, 1, 1, 0, 1, 1, 0, 0, 0]
dfiQueries['alzheimer risk'] =   [1, 1, 1, 1, 0, 1, 1, 0, 0, 0]


def MAPscore(hmap, n):
    score = []
    for k in hmap.keys():
        sums = 0
        a=0
        for i, v in enumerate(hmap[k]):
            sums += v
            a+=sums/(i+1)
        score.append(a/n)
    return sum(score)/n
    
print('tfidf: {} \nokapiBM25: {} \nDFI: {}'.format(MAPscore(tfidfQueries, 10),
                                                   MAPscore(okapiQueries, 10),
                                                   MAPscore(dfiQueries, 10)))



tfidf: 0.7776666666666666 
okapiBM25: 0.7712380952380953 
DFI: 0.7813333333333334
